# Warwick SEIR Model First Example

In this notebook we present how to use the `epimodels` module to set up an instantiation of the model built by University of Warwick, using some toy data.

*The Warwick model is built by University of Warwick.*

In [1]:
# Load necessary libraries
import numpy as np
import epimodels as em
import matplotlib
import plotly.graph_objects as go
from matplotlib import pyplot as plt
from iteration_utilities import deepflatten

## Model Setup
### Define setup matrices for the Warwick Model

In [2]:
# Populate the model
regions = ['London', 'Cornwall']
age_groups = ['0-10', '10-25']

# Initial state of the system
house_contact_data_matrix_0 = 0.2 * np.array([[10, 5.2], [0, 3]])
house_contact_data_matrix_1 = 0.2 * np.array([[1, 0], [0, 3]])

nonhouse_contact_data_matrix_0 = 0.8 * np.array([[10, 5.2], [0, 3]])
nonhouse_contact_data_matrix_1 = 0.8 * np.array([[1, 0], [0, 3]])

house_region_data_matrix_0_0 = 0.2 * np.array([[1, 10], [1, 6]])
house_region_data_matrix_0_1 = 0.2 * np.array([[0.5, 3], [0.3, 3]])
house_region_data_matrix_1_0 = 0.2 * np.array([[0.85, 1], [0.9, 6]])
house_region_data_matrix_1_1 = 0.2 * np.array([[0.5, 0.2], [0.29, 4.6]])

nonhouse_region_data_matrix_0_0 = 0.8 * np.array([[1, 10], [1, 6]])
nonhouse_region_data_matrix_0_1 = 0.8 * np.array([[0.5, 3], [0.3, 3]])
nonhouse_region_data_matrix_1_0 = 0.8 * np.array([[0.85, 1], [0.9, 6]])
nonhouse_region_data_matrix_1_1 = 0.8 * np.array([[0.5, 0.2], [0.29, 4.6]])


house_contacts_0 = em.ContactMatrix(age_groups, house_contact_data_matrix_0)
house_contacts_1 = em.ContactMatrix(age_groups, house_contact_data_matrix_1)
house_regional_0_0 = em.RegionMatrix(
    regions[0], age_groups, house_region_data_matrix_0_0)
house_regional_0_1 = em.RegionMatrix(
    regions[1], age_groups, house_region_data_matrix_0_1)
house_regional_1_0 = em.RegionMatrix(
    regions[0], age_groups, house_region_data_matrix_1_0)
house_regional_1_1 = em.RegionMatrix(
    regions[1], age_groups, house_region_data_matrix_1_1)

nonhouse_contacts_0 = em.ContactMatrix(age_groups, nonhouse_contact_data_matrix_0)
nonhouse_contacts_1 = em.ContactMatrix(age_groups, nonhouse_contact_data_matrix_1)
nonhouse_regional_0_0 = em.RegionMatrix(
    regions[0], age_groups, nonhouse_region_data_matrix_0_0)
nonhouse_regional_0_1 = em.RegionMatrix(
    regions[1], age_groups, nonhouse_region_data_matrix_0_1)
nonhouse_regional_1_0 = em.RegionMatrix(
    regions[0], age_groups, nonhouse_region_data_matrix_1_0)
nonhouse_regional_1_1 = em.RegionMatrix(
    regions[1], age_groups, nonhouse_region_data_matrix_1_1)

# Matrices contact
house_matrices_contact = [house_contacts_0, house_contacts_1]
nonhouse_matrices_contact = [nonhouse_contacts_0, nonhouse_contacts_1]
time_changes_contact = [1, 14]

house_matrices_region = [
    [house_regional_0_0, house_regional_0_1],
    [house_regional_1_0, house_regional_1_1]]
nonhouse_matrices_region = [
    [nonhouse_regional_0_0, nonhouse_regional_0_1],
    [nonhouse_regional_1_0, nonhouse_regional_1_1]]
time_changes_region = [1, 14]

### Set the parameters and initial conditions of the model and bundle everything together

In [3]:
# Instantiate model
model = em.WarwickSEIRModel()

# Set the region names, contact and regional data of the model
model.set_regions(regions)
model.set_age_groups(age_groups)
model.read_contact_data(house_matrices_contact, nonhouse_matrices_contact, time_changes_contact)
model.read_regional_data(house_matrices_region, nonhouse_matrices_region, time_changes_region)

# Set regional and time dependent parameters
regional_parameters = em.WarwickRegParameters(
    model=model,
    region_index=1,
    H=[0.8, 0.8]
)

# Set ICs parameters
ICs = em.WarwickICs(
    model=model,
    susceptibles_IC=[[1500, 600], [700, 400]],
    exposed_f_IC=[[0, 0], [0, 0]],
    exposed_sd_IC=[[0, 0], [0, 0]],
    exposed_su_IC=[[0, 0], [0, 0]],
    exposed_q_IC=[[0, 0], [0, 0]],
    detected_f_IC=[[0, 10], [2, 4]],
    detected_qf_IC=[[0, 0], [0, 0]],
    detected_sd_IC=[[10, 20], [0, 5]],
    detected_su_IC=[[20, 20], [15, 35]],
    detected_qs_IC=[[0, 0], [0, 0]],
    undetected_f_IC=[[7, 13], [14, 2]],
    undetected_s_IC=[[0, 10], [0, 0]],
    undetected_q_IC=[[0, 0], [0, 0]],
    recovered_IC=[[0, 0], [0, 0]]
)

# Set disease-specific parameters
disease_parameters = em.WarwickDiseaseParameters(
    model=model,
    tau=0.4,
    d=0.4 * np.ones(len(age_groups))
)

# Set transmission parameters
transmission_parameters = em.WarwickTransmission(
    model=model,
    epsilon=0.5,
    gamma=1,
    sigma=0.5 * np.ones(len(age_groups))
)

# Set other simulation parameters
simulation_parameters = em.WarwickSimParameters(
    model=model,
    method='RK45',
    times=np.arange(1, 20.5, 0.5).tolist()
)

# Set all parameters in the controller
parameters = em.WarwickParametersController(
    model=model,
    regional_parameters=regional_parameters,
    ICs=ICs,
    disease_parameters=disease_parameters,
    transmission_parameters=transmission_parameters,
    simulation_parameters=simulation_parameters
)

### Simulate for one of the regions: **London**

In [4]:
# Simulate for all the regions
outputs = []

for r, reg in enumerate(regions):
    # List of initial conditions and parameters that characterise the model
    parameters.regional_parameters.region_index = r + 1

    # Simulate using the ODE solver
    outputs.append(model.simulate(parameters))

## Plot the comparments of the two methods against each other
### Setup ``plotly`` and default settings for plotting

In [5]:
from plotly.subplots import make_subplots

colours = ['blue', 'red', 'green', 'purple', 'orange', 'black', 'gray', 'pink']

### Plot the comparments of the two methods against each other

In [6]:
# Trace names - represent the solver used for the simulation
trace_name = ['region {}'.format(r) for r in regions]

# Compartment list - type and age
comparments = []
for n in model.output_names():
    comparments.append('{}'.format(n))

# Plot for each comparment
for c, comparment in enumerate(comparments):
    fig = go.Figure()
    fig = make_subplots(rows=int(np.ceil(len(age_groups)/2)), cols=2, subplot_titles=tuple('ages {}'.format(a) for a in age_groups))
    # Plot (line plot for each solver method for each age)
    for a, age in enumerate(age_groups):
        if a != 0:
            for o, out in enumerate(outputs):
                fig.add_trace(
                    go.Scatter(
                        y=out[:, c*len(age_groups)+a],
                        x=parameters.simulation_parameters.times,
                        mode='lines',
                        name=trace_name[o],
                        line_color=colours[o],
                        showlegend=False
                    ),
                    row= int(np.floor(a / 2)) + 1,
                    col= a % 2 + 1
                )
        
        else:
            for o, out in enumerate(outputs):
                fig.add_trace(
                    go.Scatter(
                        y=out[:, c*len(age_groups)+a],
                        x=parameters.simulation_parameters.times,
                        mode='lines',
                        name=trace_name[o],
                        line_color=colours[o]
                    ),
                    row= int(np.floor(a / 2)) + 1,
                    col= a % 2 + 1
                )

    # Add axis labels
    fig.update_layout(
        boxmode='group',
        title=comparment, 
        width=800,
        plot_bgcolor='white',
        xaxis=dict(linecolor='black'),
        yaxis=dict(linecolor='black'),
        xaxis2=dict(linecolor='black'),
        yaxis2=dict(linecolor='black'),
        xaxis3=dict(linecolor='black'),
        yaxis3=dict(linecolor='black'),
        xaxis4=dict(linecolor='black'),
        yaxis4=dict(linecolor='black'),
        xaxis5=dict(linecolor='black'),
        yaxis5=dict(linecolor='black'),
        xaxis6=dict(linecolor='black'),
        yaxis6=dict(linecolor='black'),
        xaxis7=dict(linecolor='black'),
        yaxis7=dict(linecolor='black'),
        xaxis8=dict(linecolor='black'),
        yaxis8=dict(linecolor='black'),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        ))

    fig.show()

### Plot the super-spreader symptomatic infectious comparment for one region (London)

In [7]:
# Trace names - represent the solver used for the simulation
trace_name = ['ages {}'.format(a) for a in age_groups]

# Compartment list - type and age
fig = go.Figure()

# Plot (line plot for each method of simulation)
for a in range(len(age_groups)):
    fig.add_trace(
        go.Scatter(
            y=outputs[0][:, 8 * len(age_groups) + a],
            x=parameters.simulation_parameters.times,
            mode='lines',
            name=trace_name[a],
            line_color=colours[a]
        )
    )

# Add axis labels
fig.update_layout(
    boxmode='group',
    title='Iss in region {}'. format(regions[0]),
    plot_bgcolor='white',
    xaxis=dict(linecolor='black'),
    yaxis=dict(linecolor='black'),
    )

fig.write_image('images/Toy.pdf')
fig.show()